In [1]:
import pandas as pd
import re
import requests
import json
from tqdm import tqdm, notebook
notebook.tqdm.pandas()
from collections import Counter

C:\Users\mehdi\Anaconda3\envs\auto_tagger\lib\site-packages\tqdm\std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### خواندن دادگان از فایل

In [2]:
data = pd.read_csv('raw_data.csv')
sample = data.sample(n=10, random_state=44)
sample.head()


,author,body,comments,lastmodified,readin,slug,tags,title
383,فاطمه زنگنه,"<div class=""entry-content clearfix single-post...",[{'content': ['سلام مطالب مفید بودند ولی من فر...,2019-11-13T11:02:21+03:30,-,182804-%d9%85%d8%a7%d9%87%db%8c-%d9%87%d8%a7%d...,ترین های دنیا,۲۵ نوع از پرطرفدارترین ماهی های آکواریومی و شر...
314,حسام حسین پور,"<div class=""entry-content clearfix single-post...",[{}],2017-09-18T15:25:42+04:30,-,28358-%d8%b3%d8%a7%d8%ae%d8%aa%d8%a7%d8%b1-%d8...,"رهبری سازمانی,ساختار سازمانی",ساختار سازمانی چیست؟
74,الهام ناصری,"<div class=""entry-content clearfix single-post...",[{'content': ['دفترچه واقعا رمان زیباییه من زب...,2018-12-15T15:13:03+03:30,-,128215-%d8%a8%d9%87%d8%aa%d8%b1%db%8c%d9%86-%d...,"رابطه عاشقانه,رمان عاشقانه,معرفی رمان",بهترین رمان های عاشقانه که حس عشق را در شما زن...
67,محمد رحمانی,"<div class=""entry-content clearfix single-post...","[{'content': ['👌'], 'publish': ['1399-07-12\\1...",2018-12-15T10:39:24+03:30,-,152456-%d9%85%d8%b9%d8%b1%d9%88%d9%81-%d8%aa%d...,NaN,معروف ترین آشپزهای جهان که در حرفه‌ خود بهترین...
135,مانی حدادی,"<div class=""entry-content clearfix single-post...",[{'content': ['چرا پراید تو این لیست وجود ندار...,2018-06-23T09:30:18+04:30,-,105756-%d8%a7%db%8c%d9%85%d9%86-%d8%aa%d8%b1%d...,"تجهیزات خودرو,دانستنی های خودرو,سیستم ماشین",ایمن ترین خودروها در دنیا را بشناسید


### پردازش متون جمع آوری شده 

In [3]:
stopwords_path = 'stopwords.txt'
def eliminate_non_persian_chars(text):
    space_codepoints ='\u0020\u2000-\u200F\u2028-\u202F'
    persian_alpha_codepoints = '\u0621-\u0628\u062A-\u063A\u0641-\u0642\u0644-\u0648\u064E-\u0651\u0655\u067E\u0686\u0698\u06A9\u06AF\u06BE\u06CC'
    regex = re.compile(f'[^{persian_alpha_codepoints+space_codepoints}]')
    persian_text = regex.sub('', text)
    regex = re.compile('\s+')
    output_text = regex.sub(' ', persian_text)
    return output_text
def eliminate_stopwords(text):
    stopwords = open(stopwords_path, 'r', encoding='utf-8').read().split('\n')
    stopwords = [item.strip() for item in stopwords]
    tokenized = text.split(' ')
    output_text = []
    for token in tokenized:
        if token not in stopwords:
            output_text.append(token)
    return ' '.join(output_text)

In [4]:
data['cleaned_body'] = data.body.progress_apply(
                            lambda text:eliminate_stopwords(eliminate_non_persian_chars(text)))

### مشخص کردن عبارت های کلیدی

In [5]:
def keyphrases(text):
    # static parts 
    url = "http://api.cicap.ir/keyphrase/v1.0/rapiddetect"
    method = "POST"
    headers = {
        'Content-Type': 'text/plain'
    }
    # dynamic part 
    payload='''{\r\"param\":\"
               {\\\"lang\\\":\\\"fa\\\",\\\"docType\\\":\\\"1\\\",
               \\\"docTitle\\\":\\\"\\\",\\\"docSummary\\\":\\\"\\\",
               \\\"docBody\\\":\\\"%s\\\"}\",
               \"token\":\"11111111-1111-1111-1111-111111111111\",
               \"priority\":1,
               \"senderAddress\":\"sampleAddress.com\"}'''%(text)
    response = requests.request(method, url, headers=headers, data=payload.encode('utf-8'))
    
    try:
        data = response.json()
        data = eval(eval(data['data'])['result'])['keyPhrase']
        phrases = [item['key'] for item in data]
        return phrases
    except:
        return []
    


In [ ]:
data['key_phrases'] = data.cleaned_body.progress_apply(
                            lambda text:keyphrases(text))

### ذخیره سازی داده ها

In [10]:
# data.to_csv('new_data.csv')

22301

### خواندن دادگان برای آزمایش اصلی

In [7]:
df = pd.read_csv('proper_data.csv')
df = df[df['lastmodified']!='lastmodified']
df['bin'] = df['lastmodified'].apply(
    lambda x:' '.join(x.split('-')[:2]))
grouped = df.groupby('bin')
result = []
for group_name, df_group in tqdm(grouped):
    tags = df_group.tags.to_list()
    tags = Counter(tags)
    keyphrases = df_group.key_phrases.apply(lambda x:eval(x)).to_list()
    keyphrases = [item for sublist in keyphrases for item in sublist]
    keyphrases = Counter(keyphrases)
    keyphrases = {k: v for k, v in sorted(keyphrases.items(), 
                                          key=lambda item: item[1],
                                          reverse=True)}
    raw_tags = {}; counter=0
    for k, v in keyphrases.items():
        counter+=1
        if counter>len(keyphrases)//5 or counter>=20:
            break
        raw_tags[k]=v
    result.append([group_name, list(raw_tags.keys()), list(tags.keys())])
result_df = pd.DataFrame(result, columns =['زمان انتشار', 'برچسب های اصلی', 'برچسب های پیش بینی شده']) 

100%|█████████████████████████████████████████████████████████████████████████████████| 46/46 [00:00<00:00, 368.00it/s]


In [8]:
result_df.sample(n=10, random_state=44)

,زمان انتشار,برچسب های اصلی,برچسب های پیش بینی شده
6,2017 08,"[مشکلات جلسه مطرح جلسه, جلسه جلسه, یادآوری جلس...","[ارتباط موثر,اصول سخنرانی,طرز صحبت کردن, ارگون..."
5,2017 06,"[سلامتی کار زندگی آزادی, زندگی مرور کلیدهای مو...","[افراد موفق,ایجاد انگیزه,عبارات الهام بخش, موف..."
15,2018 06,"[زندگی تان مشاهده کتاب الکترونیکی, کنیداین کتا...","[nan, گران ترین کشورها, زنان قدرتمند,زنان موفق..."
39,2020 06,"[هدف زندگی عشق ورزیدن, هدف زندگی دوست حقیقی, پ...","[افزایش انگیزه, ذهن آرام,ذهن قدرتمند,لذت زندگی..."
31,2019 10,"[خواص ماسک, حیوانات دنیا واقعا خطرناک, فهرست خ...","[عجایب دنیا,ناشناخته های جهان, درمان بیماری ها..."
30,2019 09,"[خرید خرید صورت عمده, زمان خرید بودجه خریدتان,...","[آموزش پس انداز,برنامه ریزی مالی,صرفه جویی,مدی..."
26,2019 05,"[کیک کالری گرم, چربی گرم چربی اشباع نشده, چربی...","[دسرها,غذای کربوهیدرات دار, انواع سردرد,حالت ت..."
11,2018 02,"[آسان ترین کشورها مهاجرت ایتالیا, مهاجران پروس...","[مهاجرت آسان, قیمت خودرو, عشق ورزیدن,منتخب سرد..."
17,2018 08,"[جاذبه دیدنی نصف جهان, شادتر زندگی مجموعه صوتی...","[منتخب سردبیر,مهاجرت آسان,مهاجرت ارزان, مهاجرت..."
27,2019 06,"[فیلم خنده دار, فیلم تأثیرگذاری زندگی زنی, لیس...","[سرگرمی,فیلم سینمایی,معرفی فیلم, داستان جنگ,فی..."


In [5]:
result_df.to_excel('result.xlsx')